In [1]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

First lets create the schema excel files from the pdf descriptions:

In [2]:
from scripts.covert_reporting_structure_pdf_to_excel import main

In [3]:
main("tmp/schemas.xlsx")

We can now explor the schemas

In [4]:
from process_k203.process_report import get_schemas

In [5]:
schemas = get_schemas("tmp/schemas.xlsx")

In [6]:
print('\n'.join(list(schemas.keys())))

schema_description
assets
info
price
id
asset_country
market_type
currency
rating_agency
financial_institution
anchor_interest
anchor_interest_duration
origin_country


In [7]:
schemas["assets"].head()

מזהה נכס
1             מניה
3    יחידת השתתפות
4       יחידת מו"פ
5           זכויות
6       כתב אופציה
Name: נכסים והתחייבויות, dtype: object

Next we'll process raw data from Israel's Security Authority, apply the schemas and insert it into a database.
download data from [here](https://drive.google.com/drive/folders/1lpCv88vC4h55F-vGN-SbkoiLdGQk-f4O)

In [8]:
import zipfile
import pandas as pd
from tqdm.notebook import tqdm
from joblib import parallel_backend, delayed, Parallel

In [9]:
with zipfile.ZipFile("tmp/MutualFund.zip") as zipped:
    files = [n for n in zipped.namelist() if n.endswith(".xlsx")]
def send(excel_file):
    with zipfile.ZipFile("tmp/MutualFund.zip") as zipped:
        with zipped.open(excel_file) as f:
            return pd.read_excel(f)
dfs = [send(f) for f in tqdm(files)]

In [10]:
df = pd.concat(dfs, ignore_index=True)

In [11]:
df.shape

(9371618, 28)

In [12]:
from process_k203.process_report import apply_schemas
from process_k203.constants import SCHEMA_POSITIONS
import dask.dataframe as dd

In [13]:
df = dd.from_pandas(df, npartitions=16)

In [ ]:
processed = apply_schemas(schemas, SCHEMA_POSITIONS, df).compute()

c:\users\imri\github\open_pension\data_science\venv\lib\site-packages\dask\dataframe\core.py:3020: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Instrument Type Num', 'object'))

  warnings.warn(meta_warning(meta))
c:\users\imri\github\open_pension\data_science\venv\lib\site-packages\dask\dataframe\core.py:3020: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(fu

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine("sqlite:///db.sqlite")
processed.to_sql("k203", index=False, con=engine)